---

# **Chapter 11: Patterns in Modern Language Features**

## **Opening Context**

Software development is not static. Languages evolve, absorbing idioms and patterns that were once manually crafted. The classic Design Patterns catalog from 1994 addressed shortcomings of languages like C++ and Smalltalk. Today’s languages—C#, Java, Python, JavaScript/TypeScript—have integrated many of those concepts as first‑class language features. What once required a dozen lines of boilerplate can now be expressed with a single keyword or a lambda.

This chapter explores three major ways modern language features have transformed how we implement patterns:

1. **Dependency Injection (DI)** has become the preferred way to manage object creation and lifetimes, effectively replacing the Singleton and Factory patterns in many contexts.
2. **Lambdas and Higher‑Order Functions** allow us to replace several behavioral patterns (Strategy, Command, Template Method, Observer) with concise, inline behavior.
3. **Lazy Initialization** is now often provided by the language or its standard library, eliminating the need to hand‑roll thread‑safe lazy loading.

Understanding these evolutions helps you write cleaner, more expressive code that leverages the full power of your language, while still recognising when the classic pattern is the right tool.

---

## **11.1 Dependency Injection (DI) as the "New" Singleton/Factory**

### **The Problem with Classic Creational Patterns**

Before modern DI containers, developers frequently used the **Singleton** pattern to ensure a single instance of a class, and the **Factory** pattern to encapsulate object creation. Both have significant drawbacks:

- **Singleton** introduces global state, making unit testing difficult (you cannot easily replace the instance with a mock) and hiding dependencies (any method can call `Singleton.getInstance()`).
- **Factory** often leads to factories for every aggregate root, creating boilerplate and still coupling the client to the factory.

Consider a simple `OrderService` that needs a `PaymentGateway` and an `InventoryService`. A classic Singleton approach:

```typescript
// ❌ SINGLETON: Hidden dependencies, global state
class PaymentGateway {
  private static instance: PaymentGateway;
  private constructor() {}
  
  static getInstance(): PaymentGateway {
    if (!PaymentGateway.instance) {
      PaymentGateway.instance = new PaymentGateway();
    }
    return PaymentGateway.instance;
  }
  
  charge(amount: number): void { /* ... */ }
}

class InventoryService {
  private static instance: InventoryService;
  private constructor() {}
  
  static getInstance(): InventoryService {
    if (!InventoryService.instance) {
      InventoryService.instance = new InventoryService();
    }
    return InventoryService.instance;
  }
  
  reserve(productId: string, quantity: number): boolean { /* ... */ }
}

class OrderService {
  processOrder(order: Order): void {
    // Hidden dependencies: Who created these? When are they initialized?
    PaymentGateway.getInstance().charge(order.total);
    InventoryService.getInstance().reserve(order.productId, order.quantity);
  }
}
```

**Problems**:
- `OrderService` now has hidden dependencies on `PaymentGateway` and `InventoryService`.
- Unit testing `OrderService` is impossible without also initialising those singletons.
- The singleton constructors are private, so you cannot easily substitute mocks.

### **The Solution: Dependency Injection**

**Dependency Injection** is a pattern where an object receives its dependencies from an external source rather than creating them itself. This implements the **Dependency Inversion Principle** (DIP): high‑level modules should not depend on low‑level modules; both should depend on abstractions.

There are three common forms of DI:

1. **Constructor Injection** (most common) – dependencies are provided through the class constructor.
2. **Property/Setter Injection** – dependencies are assigned to public properties after construction.
3. **Method Injection** – dependencies are passed as arguments to specific methods.

Let's refactor the example using constructor injection:

```typescript
// ✅ DEPENDENCY INJECTION: Dependencies are explicit
interface IPaymentGateway {
  charge(amount: number): void;
}

interface IInventoryService {
  reserve(productId: string, quantity: number): boolean;
}

class PaymentGateway implements IPaymentGateway {
  charge(amount: number): void {
    console.log(`Charging $${amount}`);
  }
}

class InventoryService implements IInventoryService {
  reserve(productId: string, quantity: number): boolean {
    console.log(`Reserving ${quantity} of ${productId}`);
    return true;
  }
}

class OrderService {
  // Dependencies are injected via constructor
  constructor(
    private paymentGateway: IPaymentGateway,
    private inventory: IInventoryService
  ) {}
  
  processOrder(order: Order): void {
    this.paymentGateway.charge(order.total);
    this.inventory.reserve(order.productId, order.quantity);
  }
}

// Manual wiring (often called "poor man's DI")
const paymentGateway = new PaymentGateway();
const inventory = new InventoryService();
const orderService = new OrderService(paymentGateway, inventory);
orderService.processOrder(order);
```

**Benefits**:
- **Explicit dependencies** – the constructor signature tells you exactly what the class needs.
- **Testability** – you can pass mock implementations:

```typescript
// Unit test with mocks
const mockPayment = { charge: jest.fn() };
const mockInventory = { reserve: jest.fn().mockReturnValue(true) };
const service = new OrderService(mockPayment, mockInventory);
service.processOrder(testOrder);
expect(mockPayment.charge).toHaveBeenCalled();
```

- **Flexibility** – you can easily swap implementations (e.g., different payment gateways).

### **From Manual DI to DI Containers**

As applications grow, manually wiring dependencies becomes tedious. This is where **DI Containers** (also called IoC containers) come in. They automatically resolve and inject dependencies based on configuration.

In TypeScript, popular DI containers include **InversifyJS** and **TypeDI**. Here’s an example using InversifyJS with decorators:

```typescript
// Using InversifyJS DI Container
import 'reflect-metadata';
import { injectable, inject, Container } from 'inversify';

const TYPES = {
  PaymentGateway: Symbol.for('PaymentGateway'),
  InventoryService: Symbol.for('InventoryService'),
  OrderService: Symbol.for('OrderService')
};

@injectable()
class PaymentGateway implements IPaymentGateway {
  charge(amount: number): void { /* ... */ }
}

@injectable()
class InventoryService implements IInventoryService {
  reserve(productId: string, quantity: number): boolean { /* ... */ }
}

@injectable()
class OrderService {
  constructor(
    @inject(TYPES.PaymentGateway) private paymentGateway: IPaymentGateway,
    @inject(TYPES.InventoryService) private inventory: IInventoryService
  ) {}
  
  processOrder(order: Order): void {
    this.paymentGateway.charge(order.total);
    this.inventory.reserve(order.productId, order.quantity);
  }
}

// Container configuration
const container = new Container();
container.bind<IPaymentGateway>(TYPES.PaymentGateway).to(PaymentGateway);
container.bind<IInventoryService>(TYPES.InventoryService).to(InventoryService);
container.bind<OrderService>(TYPES.OrderService).to(OrderService);

// Resolve (the container automatically injects dependencies)
const orderService = container.get<OrderService>(TYPES.OrderService);
```

**How this replaces Singleton and Factory**:

- **Singleton replacement**: In the container, you can configure the **lifetime** of a dependency. For example, to mimic Singleton behaviour, you can use a **singleton scope**:

```typescript
container.bind<IPaymentGateway>(TYPES.PaymentGateway)
  .to(PaymentGateway)
  .inSingletonScope(); // One instance shared across all consumers
```

Now every class that needs `IPaymentGateway` receives the same instance, exactly like Singleton, but without the global state and hidden dependencies. The container manages the instance, and you can easily swap it for testing.

- **Factory replacement**: If you need to create objects with runtime parameters, you can inject a factory. Many DI containers provide auto‑generated factories:

```typescript
// Using a factory in Inversify
container.bind<interfaces.Factory<PaymentGateway>>(TYPES.PaymentGatewayFactory)
  .toFactory<PaymentGateway>((context) => {
    return () => {
      return context.container.get(TYPES.PaymentGateway);
    };
  });
```

Or you can simply inject the container itself (though this is often discouraged as it becomes a Service Locator). Better to use explicit factory interfaces.

### **DI as a Pattern, Not a Framework**

It’s important to note that Dependency Injection is first and foremost a **design pattern**, not a framework. You can practice DI without any container by manually wiring dependencies at the composition root (the entry point of your application). Frameworks like Spring (Java), ASP.NET Core (C#), and NestJS (Node.js) have built‑in DI containers that make it convenient, but the core idea remains the same: **dependencies are given to an object, not created by it**.

> **Key Insight**: DI, combined with a container, replaces the need for most custom Singleton and Factory implementations. The container becomes the single place that manages object creation and lifetimes, adhering to the **Single Responsibility Principle**.

---

## **11.2 Replacing Patterns with Lambdas and Higher‑Order Functions**

Modern languages support **first‑class functions** (lambdas, closures, delegates) that can be passed around like data. This capability allows us to replace several classic behavioral patterns with much simpler constructs.

### **Strategy Pattern → Lambda Parameter**

The classic Strategy pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable. The standard implementation requires an interface and concrete classes.

**Classic Strategy**:

```typescript
// Classic Strategy
interface SortStrategy {
  sort(data: number[]): number[];
}

class BubbleSort implements SortStrategy {
  sort(data: number[]): number[] {
    console.log('Bubble sort');
    return data.sort((a, b) => a - b);
  }
}

class QuickSort implements SortStrategy {
  sort(data: number[]): number[] {
    console.log('Quick sort');
    return data.sort((a, b) => a - b);
  }
}

class Sorter {
  constructor(private strategy: SortStrategy) {}
  
  executeStrategy(data: number[]): number[] {
    return this.strategy.sort(data);
  }
}

// Usage
const sorter = new Sorter(new QuickSort());
sorter.executeStrategy([3, 1, 2]);
```

**With Lambdas**:

If the language supports first‑class functions, the `SortStrategy` interface becomes unnecessary. We can pass the sorting function directly:

```typescript
// Functional approach: Strategy is just a function type
type SortFunction = (data: number[]) => number[];

class Sorter {
  // The strategy is a function, not an object
  constructor(private sortFn: SortFunction) {}
  
  execute(data: number[]): number[] {
    return this.sortFn(data);
  }
}

// Define strategies as lambdas
const bubbleSort: SortFunction = (data) => {
  console.log('Bubble sort');
  return data.sort((a, b) => a - b);
};

const quickSort: SortFunction = (data) => {
  console.log('Quick sort');
  return data.sort((a, b) => a - b);
};

// Use directly
const sorter = new Sorter(quickSort);
sorter.execute([3, 1, 2]);

// Or even inline
const anotherSorter = new Sorter((data) => data.sort((a,b) => a - b));
```

**Why this works**: The strategy is just a piece of behavior. By representing it as a function, we eliminate the need for a hierarchy of strategy classes. This is especially powerful in languages with type inference and concise lambda syntax.

### **Command Pattern → Function / Runnable**

The Command pattern encapsulates a request as an object, allowing parameterization, queuing, and undoable operations. With lambdas, we can replace the command object with a function.

**Classic Command**:

```typescript
interface Command {
  execute(): void;
}

class SaveCommand implements Command {
  constructor(private document: Document) {}
  
  execute(): void {
    this.document.save();
  }
}

class PrintCommand implements Command {
  constructor(private document: Document) {}
  
  execute(): void {
    this.document.print();
  }
}

// Invoker
class MenuItem {
  constructor(private command: Command) {}
  
  click(): void {
    this.command.execute();
  }
}
```

**With Lambdas**:

In languages like JavaScript/TypeScript, a command can simply be a function:

```typescript
type Command = () => void;

class MenuItem {
  constructor(private command: Command) {}
  
  click(): void {
    this.command();
  }
}

// Usage
const saveCommand: Command = () => document.save();
const printCommand: Command = () => document.print();

const saveItem = new MenuItem(saveCommand);
const printItem = new MenuItem(printCommand);
```

For commands that need parameters, you can use closures or partially applied functions.

### **Template Method Pattern → Higher‑Order Function**

The Template Method pattern defines the skeleton of an algorithm in a method, deferring some steps to subclasses. With higher‑order functions, we can pass the variable steps as arguments.

**Classic Template Method**:

```typescript
abstract class DataProcessor {
  // Template method
  process(): void {
    this.loadData();
    this.processData();
    this.saveData();
  }
  
  protected abstract loadData(): void;
  protected abstract processData(): void;
  protected abstract saveData(): void;
}

class CsvProcessor extends DataProcessor {
  protected loadData(): void { console.log('Load CSV'); }
  protected processData(): void { console.log('Process CSV'); }
  protected saveData(): void { console.log('Save CSV'); }
}
```

**With Higher‑Order Function**:

Instead of subclassing, we pass the variable behavior as functions:

```typescript
type Step = () => void;

function processData(load: Step, process: Step, save: Step): void {
  load();
  process();
  save();
}

// Usage
processData(
  () => console.log('Load CSV'),
  () => console.log('Process CSV'),
  () => console.log('Save CSV')
);
```

If some steps have default implementations, we can use partial application or default parameters.

### **Observer Pattern → Event Emitters / Callbacks**

The Observer pattern defines a one‑to‑many dependency so that when one object changes state, all its dependents are notified. Modern languages often provide built‑in event handling or can be replaced with simple callback lists.

**Classic Observer**:

```typescript
interface Observer {
  update(data: any): void;
}

class Subject {
  private observers: Observer[] = [];
  
  attach(observer: Observer): void {
    this.observers.push(observer);
  }
  
  notify(data: any): void {
    for (const obs of this.observers) {
      obs.update(data);
    }
  }
}

class ConcreteObserver implements Observer {
  update(data: any): void {
    console.log('Received:', data);
  }
}
```

**With Callbacks (Lambdas)**:

Instead of requiring an `Observer` interface, we can maintain a list of callback functions:

```typescript
class Subject {
  private observers: Array<(data: any) => void> = [];
  
  attach(callback: (data: any) => void): void {
    this.observers.push(callback);
  }
  
  notify(data: any): void {
    this.observers.forEach(cb => cb(data));
  }
}

// Usage
const subject = new Subject();
subject.attach((data) => console.log('Observer 1:', data));
subject.attach((data) => console.log('Observer 2:', data));
subject.notify('Hello');
```

This approach is simpler and more flexible. Many modern frameworks use this pattern (e.g., Node.js `EventEmitter`, RxJS Observables).

### **Trade‑offs and Considerations**

While replacing patterns with lambdas reduces boilerplate, there are cases where the classic pattern is still preferable:

- **Complex behavior** that requires multiple methods (e.g., a Strategy that needs initialization and cleanup) might be better encapsulated in a class.
- **Discoverability**: Named classes can be easier to find and document than anonymous lambdas.
- **Framework integration**: Some frameworks expect classes (e.g., Angular’s dependency injection works best with classes).
- **Performance**: In some languages, creating many lambdas can have a higher overhead than reusing strategy objects.

The key is to **use lambdas when the behavior is simple and the context is clear**, but don't force a functional style when a class would be more expressive.

---

## **11.3 Lazy Loading and the Lazy Initialization Pattern**

**Lazy loading** (or lazy initialization) is the tactic of delaying the creation of an object, the calculation of a value, or some other expensive process until the first time it is needed. This can improve startup performance and reduce memory usage.

### **Classic Lazy Initialization**

The traditional implementation checks a flag or null before creating the object:

```typescript
class ExpensiveResource {
  constructor() {
    console.log('ExpensiveResource created');
  }
  
  doWork(): void { /* ... */ }
}

class Service {
  private resource: ExpensiveResource | null = null;
  
  getResource(): ExpensiveResource {
    if (!this.resource) {
      this.resource = new ExpensiveResource();
    }
    return this.resource;
  }
  
  useResource(): void {
    this.getResource().doWork();
  }
}
```

**Problems**:
- **Not thread‑safe** – if two threads call `getResource` simultaneously, you might create two instances.
- **Boilerplate** – repeated null‑check code in every lazy property.

### **Language‑Integrated Lazy Loading**

Many modern languages provide built‑in support for lazy initialization, handling thread safety and reducing boilerplate.

#### **C#: `Lazy<T>`**

```csharp
using System;

public class Service
{
    private Lazy<ExpensiveResource> _resource = 
        new Lazy<ExpensiveResource>(() => new ExpensiveResource());
    
    public void UseResource()
    {
        _resource.Value.DoWork(); // First access triggers creation
    }
}
```

`Lazy<T>` ensures thread‑safe creation by default (configurable) and provides a simple `.Value` property.

#### **Java: `Supplier` with double‑checked locking or `AtomicReference`**

While Java has no built‑in `Lazy<T>`, libraries like Guava provide `Supplier` and `Suppliers.memoize()`. Modern Java can use `ConcurrentHashMap` for per‑key lazy loading.

#### **Python: `property` with caching**

Python’s `@property` decorator can be combined with a private attribute to create a lazy property:

```python
class Service:
    def __init__(self):
        self._resource = None
    
    @property
    def resource(self):
        if self._resource is None:
            print("Creating expensive resource")
            self._resource = ExpensiveResource()
        return self._resource

    def use_resource(self):
        self.resource.do_work()
```

For thread‑safety, you can use locking, but Python’s GIL simplifies single‑threaded cases.

#### **JavaScript/TypeScript: No built‑in, but easy to implement**

JavaScript doesn’t have a standard `Lazy`, but you can create a helper:

```typescript
class Lazy<T> {
  private instance: T | null = null;
  
  constructor(private factory: () => T) {}
  
  get value(): T {
    if (this.instance === null) {
      this.instance = this.factory();
    }
    return this.instance;
  }
}

// Usage
const resource = new Lazy(() => new ExpensiveResource());
resource.value.doWork(); // created on first use
```

For thread‑safety in Node.js (single‑threaded), this is sufficient. For web workers, you'd need locking or use `Atomics`.

### **Lazy Loading with Proxies (ES6)**

JavaScript’s `Proxy` can be used to create lazily‑initialized objects that intercept property access:

```typescript
function lazyProxy<T extends object>(factory: () => T): T {
  let instance: T | null = null;
  return new Proxy({} as T, {
    get(target, prop) {
      if (!instance) {
        instance = factory();
      }
      return instance[prop];
    }
  });
}

// Usage
const resource = lazyProxy(() => new ExpensiveResource());
resource.doWork(); // first access triggers creation
```

This approach creates a transparent lazy object: the client code doesn’t need to call `.value`; it just uses the object normally. However, it can have performance overhead and may not work with all features (e.g., `instanceof` checks).

### **Use Cases for Lazy Loading**

1. **Expensive resource initialization** – database connections, configuration parsing, large data structures.
2. **Circular dependencies** – in DI scenarios, lazy loading can break circular references by delaying creation.
3. **Conditional execution** – if a feature may never be used, loading it lazily saves resources.
4. **Startup time optimization** – defer non‑essential work until after the application is responsive.

### **Thread Safety Considerations**

In multi‑threaded environments, lazy initialization must be thread‑safe to avoid creating multiple instances. Built‑in solutions like `Lazy<T>` in C# offer various thread‑safe modes. In Java, you might use `AtomicReference` or double‑checked locking with `volatile`. In languages without built‑in support, you can use locks or concurrent collections.

> **Note**: Lazy loading can introduce subtle bugs if not implemented correctly, especially in concurrent scenarios. Prefer language‑provided or well‑tested library implementations when available.

---

## **Chapter Summary**

This chapter examined how modern language features have evolved to incorporate or replace classic design patterns:

1. **Dependency Injection** has become the standard way to manage object creation and lifetimes. By inverting control, it makes dependencies explicit, improves testability, and—when combined with a DI container—can replace the need for hand‑crafted Singleton and Factory patterns. DI containers provide flexible lifetime management (singleton, scoped, transient) and auto‑wiring, centralizing object creation in a maintainable way.

2. **Lambdas and higher‑order functions** allow us to replace several behavioral patterns with concise, inline behavior:
   - **Strategy** → function parameter
   - **Command** → function/delegate
   - **Template Method** → higher‑order function
   - **Observer** → callback list
   This shift reduces boilerplate and leverages the expressive power of modern languages. However, classic patterns still have their place when behavior is complex or when working within class‑centric frameworks.

3. **Lazy Initialization** is now often provided by language runtimes or standard libraries (`Lazy<T>` in C#, property caching in Python, or custom helpers in JavaScript). These built‑ins handle thread safety and reduce repetitive null‑check code, making lazy loading both safer and easier to implement.

**Key Insight**: The evolution of languages does not make design patterns obsolete—it internalizes them. The patterns themselves remain valuable as conceptual tools; they simply manifest differently in code. Understanding both the classic form and the modern adaptation equips you to write code that is idiomatic, maintainable, and aligned with your language’s strengths.

---

## **Next Chapter Preview**

**Chapter 12: Functional Programming Patterns (Monads, Functors, Immutability, Memoization, Currying)**

We will dive into the world of functional programming, exploring patterns that originate from category theory and have become mainstream in languages like Scala, F#, and even JavaScript. You will learn how **immutability** simplifies state management, how **functors** and **monads** (like `Option` and `Either`) provide elegant error handling, and how **memoization** and **currying** enable powerful code reuse and optimization. These patterns offer a different mindset for composing software, and they work hand‑in‑hand with the modern language features we’ve just discussed.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. the_classic_catalog/10. state_management.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='12. functional_programming_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
